This program starts with a list of companies (with CIK codes),
in this case, S&P 500 companies, gets a list of all 10-K filings
for these companies from Edgar, then gets the 10-Ks themselves in HTML format,
and finally extracts Item 7 from these filings.

Murat Aydogdu 
May 1, 2018

I started from a tutorial 
http://community.mis.temple.edu/zuyinzheng/files/2016/07/Tutorial-2-Extracting-Data-from-10-K.pdf
and modified the code substantially.

Version 02: instead of saving all of the 10K as an HTML file, just save the Item7 
porsion of it. There will be the HTML version of the original filing and sentences from it saved in a TXT file.

In [1]:
import os, csv, urllib2, time, re 
from bs4 import BeautifulSoup   
import pandas as pd

In [2]:
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')
import nltk
from nltk.tokenize import sent_tokenize
#nltk.download('punkt')

In [3]:
os.chdir('/Users/murataydogdu/Desktop/TextualAnalysis/Version1_2018June_Programs')
!pwd

In [4]:
# Step 1: Read in the ticker symbols

tickerdf = pd.read_csv('data/SP500Tickers_20180501.csv')
#CIK codes come in as integers, create a ten-character CIK code padded with zeros
tickerdf['CIK10'] = tickerdf['CIK'].astype(str).str.zfill(10) 
tickerdf.drop(['CIK'], axis=1, inplace = True)
display(tickerdf)

,Ticker,Security,CIK10
0,MMM,3M Company,0000066740
1,ABT,Abbott Laboratories,0000001800
2,ABBV,AbbVie Inc.,0001551152
3,ACN,Accenture plc,0001467373
4,ATVI,Activision Blizzard,0000718877
5,AYI,Acuity Brands Inc,0001144215
6,ADBE,Adobe Systems Inc,0000796343
7,AMD,Advanced Micro Devices Inc,0000002488
8,AAP,Advance Auto Parts,0001158449
9,AES,AES Corp,0000874761


In [ ]:
# Step 2: Obtain the links for all 10-K filings for all the firms in the list,
# then write these links into a csv file.
# Note: These links will point to an index location.
# We still need to search the HTML file that has the 10-K in that location.
# This takes several minutes

with open('data/Links.csv', 'wt') as f:
    writer = csv.writer(f)
    writer.writerow(['CIK10', 'Ticker', 'Security', 'DocIndex', 'DocLink', 'Description', 'FilingDate'])
    for index, row in tickerdf.iterrows():
        urlLink = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK="+row['CIK10']+"&type=10-K&dateb=&owner=include&count=100"

        pageRequest = urllib2.Request(urlLink)
        pageOpen = urllib2.urlopen(pageRequest)
        pageRead = pageOpen.read()

        soup = BeautifulSoup(pageRead,"html.parser")
        FormType = '10-K'
        try:
            table = soup.find("table", { "class" : "tableFile2" })
        except:
            print "No 10-K filings found for"+row['CIK10']
        DocIndex = 1
        for rows in table.findAll("tr"):
            cells = rows.findAll("td")
            if len(cells)==5:
                if cells[0].text.strip() == FormType:
                    link = cells[1].find("a",{"id": "documentsbutton"})
                    DocLink = "https://www.sec.gov"+link['href']
                    Description = cells[2].text.encode('utf8').strip() 
                    FilingDate = cells[3].text.encode('utf8').strip().replace("-","_")
                    writer.writerow([row['CIK10'], row['Ticker'], row['Security'], DocIndex, DocLink, Description, FilingDate])
                DocIndex = DocIndex + 1

In [5]:
# Step 3: Read in the 10-K links from the CSV file that was created above.
# These links will be visited one by one to search for the 10-Ks
linksdf = pd.read_csv('data/Links.csv')
display(linksdf)

,CIK10,Ticker,Security,DocIndex,DocLink,Description,FilingDate
0,66740,MMM,3M Company,1,https://www.sec.gov/Archives/edgar/data/66740/...,"Annual report [Section 13 and 15(d), not S-K I...",2018_02_08
1,66740,MMM,3M Company,2,https://www.sec.gov/Archives/edgar/data/66740/...,"Annual report [Section 13 and 15(d), not S-K I...",2017_02_09
2,66740,MMM,3M Company,3,https://www.sec.gov/Archives/edgar/data/66740/...,"Annual report [Section 13 and 15(d), not S-K I...",2016_02_11
3,66740,MMM,3M Company,4,https://www.sec.gov/Archives/edgar/data/66740/...,"Annual report [Section 13 and 15(d), not S-K I...",2015_02_12
4,66740,MMM,3M Company,5,https://www.sec.gov/Archives/edgar/data/66740/...,"Annual report [Section 13 and 15(d), not S-K I...",2014_02_13
5,66740,MMM,3M Company,6,https://www.sec.gov/Archives/edgar/data/66740/...,"Annual report [Section 13 and 15(d), not S-K I...",2013_02_14
6,66740,MMM,3M Company,7,https://www.sec.gov/Archives/edgar/data/66740/...,"[Cover]Annual report [Section 13 and 15(d), no...",2012_02_16
7,66740,MMM,3M Company,8,https://www.sec.gov/Archives/edgar/data/66740/...,"Annual report [Section 13 and 15(d), not S-K I...",2011_02_16
8,66740,MMM,3M Company,9,https://www.sec.gov/Archives/edgar/data/66740/...,"Annual report [Section 13 and 15(d), not S-K I...",2010_02_16
9,66740,MMM,3M Company,10,https://www.sec.gov/Archives/edgar/data/66740/...,"Annual report [Section 13 and 15(d), not S-K I...",2009_02_17


In [6]:
time.ctime()

'Mon Jun  4 16:23:28 2018'

In [ ]:
nbDocPause = 10 ### <=== Type your number of documents to download in one batch
nbSecPause = 1 ### <=== Type your pausing time in seconds between each batch
htmlSubPath = "HTML/" #<===The subfolder with the 10-K files in HTML format

i = 1
for index, row in linksdf.iterrows():
    if index >= 8751:
        CIK10,Ticker,Security,DocIndex,DocLink,Description,FilingDate = row
        DocIndex = "{0:02d}".format(DocIndex)
        print index, DocLink
        pageRequest = urllib2.Request(DocLink)
        pageOpen = urllib2.urlopen(pageRequest)
        pageRead = pageOpen.read()
        try:
            soup = BeautifulSoup(pageRead,"html.parser")
        except:
            print ('Problem:', index, DocLink)

        try:
            table = soup.find("table", { "summary" : "Document Format Files" })
        except:
            print "No tables found for link " + DocLink

        for row in table.findAll("tr"):
            cells = row.findAll("td")
            if len(cells)==5:
                if cells[3].text.strip() == '10-K':
                    link = cells[2].find("a")
                    #print link['href']
                    FormLink = "https://www.sec.gov"+link['href']
                    #print FormLink
                    pRequest = urllib2.Request(FormLink)
                    pOpen = urllib2.urlopen(pRequest)
                    pRead = pOpen.read()

                    htmlname = Ticker+"_"+DocIndex+"_"+FilingDate+".htm"
                    htmlpath = htmlSubPath+htmlname
                    extractItem7(pRead, htmlpath)

            if i%nbDocPause == 0:
                time.sleep(float(nbSecPause))
        i=i+1

In [ ]:
time.ctime()

In [ ]:
# Remove tables
# All the way to Item 8
def extractItem7(page, htmlpath):

    #Pre-processing the html content by removing extra white space and combining then into one line.
    page = page.strip()  #<=== remove white space at the beginning and end
    page = page.replace('\n', ' ') #<===replace the \n (new line) character with space
    page = page.replace('\r', '') #<===replace the \r (carriage returns -if you're on windows) with space
    page = page.replace('&nbsp;', ' ') #<===replace "&nbsp;" (a special character for space in HTML) with space. 
    page = page.replace('&#160;', ' ') #<===replace "&#160;" (a special character for space in HTML) with space.
    while '  ' in page:
        page = page.replace('  ', ' ') #<===remove extra space

    #Using regular expression to extract texts that match a pattern
    regexs = ('bold;\">\s*Item 7\.(.+?)bold;\">\s*Item 8\.',   #<===pattern 1: with an attribute bold before the item subtitle
              'b>\s*Item 7\.(.+?)b>\s*Item 8\.',               #<===pattern 2: with a tag <b> before the item subtitle
              'Item 7\.\s*<\/b>(.+?)Item 8\.\s*<\/b>',         #<===pattern 3: with a tag <\b> after the item subtitle          
              'Item 7\.\s*Management\.\s*<\/b(.+?)Item 8\.\s*Financial\.\s*<\/b') #<===pattern 4: with a tag <\b> after the item+description subtitle 

    for regex in regexs:
        match = re.search (regex, page, flags=re.IGNORECASE)  #<===search for the pattern in HTML using re.search from the re package. Ignore cases.

        if match:
            try:
                # Write the Item 7 portion of 10K into as an HTML file
                soup = BeautifulSoup(match.group(1), "html.parser") #<=== match.group(1) returns the texts inside the parentheses (.*?) 
                #print soup.prettify()
                htmlfile = open(htmlpath,'w')
                htmlfile.write(soup.prettify())
                htmlfile.close()

                break  
            except:
                print ('Problem:', htmlpath)

    return